In [10]:
import datetime
from dateutil.relativedelta import relativedelta

In [26]:
def create_dates_list(start_date, finish_date = None,months_delta = 3):
    
    start_date = datetime.datetime.strptime(start_date, "%d/%m/%Y").date()

    if finish_date is not None:
        finish_date = datetime.datetime.strptime(finish_date, "%d/%m/%Y").date()
    else:
        finish_date = datetime.date.today()
    
    dates_list =  []
    # current_date = start_date
    while start_date < finish_date:
        
        end_date = start_date + relativedelta(months=months_delta)
        dates_list.append([start_date.strftime("%d/%m/%Y"),end_date.strftime("%d/%m/%Y")])
        start_date = start_date + relativedelta(months=months_delta)
    return dates_list


create_dates_list('01/01/2024')

[['01/01/2024', '01/04/2024'],
 ['01/04/2024', '01/07/2024'],
 ['01/07/2024', '01/10/2024'],
 ['01/10/2024', '01/01/2025'],
 ['01/01/2025', '01/04/2025'],
 ['01/04/2025', '01/07/2025'],
 ['01/07/2025', '01/10/2025']]

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
sys.path.append('../../')
# get repo root actual dir (for file reading)
from helpers.system_helpers.repo_root import REPO_ROOT
from helpers.data_helpers.ecmwf_helpers import linear_approximation,get_wind_angle

xr_raw = xr.open_dataset(f'{REPO_ROOT}/data/doronell_data/doronell_ecmwf.grib', engine="cfgrib")
df_v100 = xr_raw['v100'].to_dataframe().reset_index()
df_u100 = xr_raw['u100'].to_dataframe().reset_index()

# get point of wind at wind farm location
df_gcmwf = linear_approximation(df_v100,'v100')
df_gcmwf = df_gcmwf.merge(linear_approximation(df_u100,'u100'), on = ['time'])
# get actual wind speed using pythag
df_gcmwf['h100'] = np.sqrt(df_gcmwf['v100']**2 + df_gcmwf['u100']**2)
# get wind direction using pythag
df_gcmwf = get_wind_angle(df_gcmwf)

min_time_str = df_gcmwf.time.min().strftime(format = '%H-%d/%m/%Y')
max_time_str = df_gcmwf.time.max().strftime(format = '%H-%d/%m/%Y')

df_gcmwf.to_feather(f'{REPO_ROOT}/data/doronell_dataweather/{min_time_str}_{max_time_str}_ws.feather')

skipping variable: paramId==49 shortName='fg10'
Traceback (most recent call last):
  File "c:\Users\EAFar\OneDrive\Documents_Alexander\repos\elec_grid\.env_eg\Lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "c:\Users\EAFar\OneDrive\Documents_Alexander\repos\elec_grid\.env_eg\Lib\site-packages\cfgrib\dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1704067200, 1704070800, 1704074400, ..., 1757836800, 1757840400,
       1757844000], shape=(14939,))) new_value=Variable(dimensions=('time',), data=array([1704045600, 1704088800, 1704132000, ..., 1757743200, 1757786400,
       1757829600], shape=(1246,)))
c:\Users\EAFar\OneDrive\Documents_Alexander\repos\elec_grid\.env_eg\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not dec

TypeError: DataFrame.to_feather() missing 1 required positional argument: 'path'

In [52]:
min_time_str = df_gcmwf.time.min().strftime(format = '%H-%d%m%Y')
max_time_str = df_gcmwf.time.max().strftime(format = '%H-%d%m%Y')
df_gcmwf.to_feather(f'{REPO_ROOT}/data/doronell_data/weather/{min_time_str}_{max_time_str}_ws.feather')

In [47]:
df_gcmwf.time.max()

Timestamp('2025-09-14 10:00:00')

In [48]:
min_time_str

'00-01/01/2024'